# Language Generation using Pretrained Models

In this notebook, we will finally look at language generation!

## Preparations
Import packages, and customize settings:

In [ ]:
# Libraries for deep learning. In the background, we will use torch / pytorch
import torch
import torch.nn.functional as F

# Libraries from huggingface to easily interact with pretrained models
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# general Python libraries:
import pandas as pd

In [ ]:
# make sure the entire text is output:
pd.set_option('display.max_colwidth', -1)

In [ ]:
# what device is this notebook running on?
device = "cuda" if torch.cuda.is_available() else "cpu"

For this notebook, we will use the `GPT2` model, that has been open-sources by openAI. We use the corresponding tokenizer and causal language model:

In [ ]:
model_name = "gpt2"
# if you have a powerful computer and want to use a larger model:
# model_name = "gpt2-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

## Greedy Search Decoding

We start with an example for greedy decoding. While there is an easy way of using this decoding strategy via the `generate` function of the `model` with the pretrained parameters, we start with a more granular look at the sampling method.

In [ ]:
input_txt = "Transformers are the"
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)

### An in-depth Look at Greedy Search
Starting from the input text "Transformers are the", we repeatedly call the model. As a result of the model, we get (among other things) the logits, the non-normalized scores by the model for each of the tokens. We then normalize these scores to probabilities, and store the tokens with the highest probabilities.

In [ ]:
iterations = []
n_steps = 8
choices_per_step = 5

with torch.no_grad():
    for _ in range(n_steps):
        iteration = dict()
        iteration["Input"] = tokenizer.decode(input_ids[0])
        output = model(input_ids=input_ids)
        # Select logits of the first batch and the last token and apply softmax
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
        # Store tokens with highest probabilities
        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice = (
                f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)"
            )
            iteration[f"Choice {choice_idx+1}"] = token_choice
        # Append predicted next token to input
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iterations.append(iteration)

Now let us look at the most probable tokens of every iteration step:

In [ ]:
pd.DataFrame(iterations)

### Using the `generate()` Function
The Huggingface transformer model has a function `generate()` to generate texts, and allows us to specify the methods to be used for the text generation. Without futher arguments, the greedy search is implemented:

In [ ]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False,
                        pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(output[0]))

We can also try to reproduce the unicorn story presented along with GPT-2:

In [ ]:
max_length = 128
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False,
                               pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(output_greedy[0]))

In [ ]:
tokenizer.decode(output_greedy[0])

The text is different from what openAI has reported when releasing GPT-2, but at first sight, it seems plausible and is grammatically correct. The involved researcher gets a name, and he works at a different university, however also in California. The later part of the generated text are very repretitive - looks like GPT-2 is highly convinced that unicorns are very intelligent :-)

### GPT as Calculator?
We can also try to make GPT-2 calculate:

In [ ]:
max_length_math = 70
input_txt_math1 = """
5 + 8 = 13
2 + 7 = 9
13 - 5 = 8
2 * 5 = 10
5 + 7 =
"""
input_ids = tokenizer(input_txt_math1, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length_math,
                               do_sample=False,
                               pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(output_greedy[0]))

Unfortunately,this is completely wrong. Even when we're restricting ourselves to the addition, our model completely fails:

In [ ]:
input_txt_math2 = """
5 + 8 = 13
2 + 7 = 9
5 + 7 =
"""
input_ids = tokenizer(input_txt_math2, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length_math,
                               do_sample=False)
print(tokenizer.decode(output_greedy[0]))

It's been known for a while that the large language models are bad in doing calcations. We revert to the fiction story about the Andine unicors.

## Beam Search
Next, we look at the beam search strategy, which keeps a set of most probable partial solutions.

First, we define two functions to compute the log-probability of a sequence from the logits we get from the model:

In [ ]:
def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

In [ ]:
def sequence_logprob(model, labels, input_len=0):
    with torch.no_grad():
        output = model(labels)
        log_probs = log_probs_from_logits(
            output.logits[:, :-1, :], labels[:, 1:])
        seq_log_prob = torch.sum(log_probs[:, input_len:])
    return seq_log_prob.cpu().numpy()

Let us calculate the log-probability of the greedy output we've obtained before:

In [ ]:
logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f"\nlog-prob: {logp:.2f}")

Now, let's generate a text continuation using beam search:

In [ ]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5,
                             do_sample=False)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

Tracking several potential continuations of the start sentence, we get a sentence that has a higher overall probability, and also sounds much more natural - but the part that the unicorns communicate in a way similar to that of human speech is still there. We can force the `generate` function not to produce repeated `n`-grams (i.e., combinations of `n` words that occurr more than once):

In [ ]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5,
                             do_sample=False, no_repeat_ngram_size=2)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

Note that the affiliation of the researcher has changed - "University of California, San Diego, and the University of California, Santa Cruz" (as was stated in the previous sequence) contains the 3-gram "University of California" twice. Since we have told the model not to repeat any 2-grams, "University of California" must only appear once.

While the model has thus correctly followed the rules we've imposed, the text - even though it might sound convincing - does not really make sense: If the unicorns speak perfect English, why would the "words of the unicorns" have to be translated into English? Also, the statement "This is the first time that we have translated a language into an English language" which is attributed to the NSF professor of linguistics and evolutionary biology, is clearly wrong.

## Sampling Methods

In order to obtain some more interesting texts, we now look at sampling methods. First, we vary the temperature:

In [ ]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True,
                             temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))

In [ ]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True,
                             temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))

Setting the temperature to 2.0 (in the first attempt) results in a very confuse text, using words that are very rare in the context, and ignoring almost all rules of grammar. With a lower temperature of 0.5, however, we get a pretty consistent and plausible text.

### Top-k Sampling
Next, we limit the choice to the top 50 tokens in every step.

In [ ]:
output_topk = model.generate(input_ids, max_length=max_length, do_sample=True,
                             top_k=50)
print(tokenizer.decode(output_topk[0]))

### Top-p Sampling
As the value `k` in the previous example was rather random, we also want to try the dynamic cut-off using top-p (or nucleus) sampling.

In [ ]:
output_topp = model.generate(input_ids, max_length=max_length, do_sample=True,
                             top_p=0.90)
print(tokenizer.decode(output_topp[0]))

## Conclusion

After tying out several approaches - what is the best one? Unfortunately, there is no universal answer. As we have seen, lower temperatures (or a deterministic approach as a limit behaviour) produces more predictable texts, at the risk of repetitions. For more creativity, increase the temperature, possibly in combination with top-k or a dynamic cutoff using top-p sampling.